In [1]:
import pandas as pd
import numpy as np
from lxml import etree
import xml.etree.ElementTree as ET
import time
import math

In [2]:
xml_positions = etree.parse('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##arbeitsplatop

In [3]:
kpimerged = pd.read_excel('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') #arbeit

In [4]:
path='C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/'    #arbeit

In [ ]:
path='C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/'    #Privat

In [ ]:
xml_positions = etree.parse('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##privat
kpimerged = pd.read_excel('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') ##privat

In [5]:
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)

In [6]:
class DataReader:
  """
  Main class to read in xml files provided by the DFB.

  Returns:
      ElementTree root: The root of the xml document.

  """

  def __init__(self,xml_file):
    self.xml_root = self._load_data(xml_file)


  def _load_data(self,xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root

  def create_dataframe(self,df_cols=None,findall_string=".//Object",time_search=None):
    """
    Create a dataframe from the xml inputs

    Args:
        df_cols (list): Optional, the column names. If no names given, columns
          detected automatically
        findall_string (str): The xpath string to iterate trees (https://www.w3schools.com/xml/xml_xpath.asp)
        time_search (float): Will currently look at parent if it's an Event, it
          will compare the time_search to the EventTime. I.e. this is a means of getting
          all passes before the frame time.

    Returns:
        pd.dataframe
    """

    populate_column_names=False
    if df_cols is None:
        df_cols = []
        populate_column_names=True

    out_df = pd.DataFrame(columns = df_cols)

    parent_map = dict((c, p) for p in self.xml_root.getiterator() for c in p)
    for node in self.xml_root.findall(findall_string):
        if time_search is not None:
          if (parent_map[node].tag != "Event"): continue
          event_time = parent_map[node].attrib["EventTime"]
          ts = ciso8601.parse_datetime(event_time)
          # to get time in seconds:
          seconds=time.mktime(ts.timetuple())
          # If timestamp is larger than the search, exit
          if seconds > time_search: continue

        # If column names are not provided, get them
        if populate_column_names==True:
            for key,value in node.attrib.items():
                df_cols.append(key)
                populate_column_names=False

        res = []
        for c in df_cols:
            res.append(node.attrib.get(c))
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index=True)
    return out_df


In [7]:
def get_tracking_data(xml_positions, half='firstHalf'):
        """Returns a dataframe with the tracking (position: x, y, z) of all players + the ball
        
        Dropping rows where ball positions is NAN

        +-------+-----------+-----------+
        |       |   BALL    |  TeamId   |
        +       +-----------+-----------+
        |       |  BallId   | PlayerId  |
        + N | T +---+---+---+---+---+---+
        |       | X | Y | Z | X | Y | A |
        +=======+===+===+===+===+===+===+
        | 0 | 0 |123|456|789|012|456|789|
        +-------+-----------+-----------+

        :param xml_positions: the parsed xml tracking data (lxml object)
        :param half: the description of the half: either "firstHalf" or "secondHalf", etc.
        :return: a dataframe containing the tracking data: position of all players and the ball
        """
        def change_type(df, frameset):
            """
            change type of columns
            different columns names and tpyes for the ball and for the rest of the players
            """
            if frameset.attrib.get('TeamId') == 'BALL':
                return df.astype(
                    # change types of columns for the ball
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        'Z':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                        'BallPossession':int, 
                        'BallStatus':int,
                    }, 
                    errors='raise'
                )
            else:
                return df.astype(
                    # change types of columns for the players
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                    }, 
                    errors='raise'
                )
        

        def create_columns(df, frameset):
            """A utility function to create a multiindex columns"""
            person_id = frameset.attrib.get('PersonId')
            team_id = frameset.attrib.get('TeamId')
            df.columns = pd.MultiIndex.from_product(
                [[team_id], [person_id], df.columns],
                names=['TeamId', 'PersonId', 'Position']
            )
            return df

        tracking_players = pd.concat([
            pd.DataFrame.from_records(
                [dict(frame.attrib) for frame in frameset.getchildren()]
            ).pipe(change_type, frameset=frameset)
             .set_index(['N', 'T'])
             .pipe(create_columns, frameset=frameset)
            for frameset in xml_positions.xpath(F'//Positions/FrameSet[@GameSection = "{half}"]')
        ], axis=1, sort=False)
        
        # this is casting the 'T' values in the index to datetime with the correct time zone
        tracking_players.index.set_levels(pd.to_datetime(tracking_players.index.get_level_values(1)), level=1, inplace = True)

        return tracking_players

In [8]:
positions=get_tracking_data(xml_positions)     
##create dataframe from xml input

In [9]:
CUID1 = 'DFL-CLU-000N8Y'
cuid1_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID1)
cuid1_team_data = DataReader(path+cuid1_player_info)
cuid1_team_meta_df = cuid1_team_data.create_dataframe()
cuid1_player_IDs = cuid1_team_meta_df.ObjectId.tolist()

In [10]:
CUID2 = 'DFL-CLU-000N8Z'
cuid2_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID2)
cuid2_team_data = DataReader(path+cuid2_player_info)
cuid2_team_meta_df = cuid2_team_data.create_dataframe()
cuid2_player_IDs = cuid2_team_meta_df.ObjectId.tolist()

In [199]:
def directionplayframe(frame,teamid):
    ###Check for teamid
    if teamid == CUID1:
        gk_df=cuid1_team_meta_df.loc[cuid1_team_meta_df['PlayingPositionEnglish']=='goalkeeper'] #select goalkeeper information from team with clubid1
        gks=gk_df['ObjectId'] #select only the ids
        frame1=half['1st half'] #define start of first half
        frame2=half['2nd half'] #define start of second half
        if frame<frame2:  #if the selected frame is smaller than frame2 the frame lies within the first half 
            for items in gks.iteritems():  # iterating through the list of goalkeeper
                if (teamid,items[1],'X') in positions.columns:  #if the id is part of the tracking data column names then continue
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])] 
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'left to right'
                        return dirofplay
                    else:
                        dirofplay = 'right to left'
                        return dirofplay
        else:
            for items in gks.iteritems():
                if (teamid,items[1],'X') in positions.columns:
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'right to left'
                        return dirofplay
                    else:
                        dirofplay = 'left to right'
                        return dirofplay
    if teamid == CUID2:
        gk_df=cuid2_team_meta_df.loc[cuid2_team_meta_df['PlayingPositionEnglish']=='goalkeeper']
        gks=gk_df['ObjectId']
        frame1=half['1st half']
        frame2=half['2nd half']
        if frame<frame2:
            for items in gks.iteritems():
                if (teamid,items[1],'X') in positions.columns:
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'left to right'
                        return dirofplay
                    else:
                        dirofplay = 'right to left'
                        return dirofplay
        else:
            for items in gks.iteritems():
                if (teamid,items[1],'X') in positions.columns:
                    gkpos=positions.loc[(slice(frame1,frame2),),(slice(None),items[1],['X','Y'])]
                    if gkpos[teamid,items[1],'X'].mean() < 0:
                        dirofplay = 'right to left'
                        return dirofplay
                    else:
                        dirofplay = 'left to right'
                        return dirofplay

In [202]:
half={} # frames der kickoff zeiten in frames
kickoffs=kpimergednew[(kpimergednew['SUBTYPE']=='Kickoff')].index.tolist()
for i in kickoffs:
    if i != 0:
        framedif=kpimergednew.iloc[i].N -kpimergednew.iloc[i-2].N
        if framedif > 10000:
            half['2nd half']=kpimergednew.iloc[i].N
    else:
        half['1st half']=kpimergednew.iloc[i].N

In [257]:
CUID1df = positions.loc[:,(CUID1,slice(None),"X")]

In [60]:
playerid='DFL-OBJ-0027G6'
dfa=Germany[CUID1,playerid,"A"][Germany[CUID1,playerid,'A']>4]
dfs=Germany[CUID1,playerid,"S"][Germany[CUID1,playerid,'A']>4]

In [41]:
def getplayername(playerid):
    if playerid in cuid1_player_IDs:
        name=cuid1_team_meta_df["Name"][cuid1_team_meta_df["ObjectId"]==playerid]
    else:
        name=cuid2_team_meta_df["Name"][cuid2_team_meta_df["ObjectId"]==playerid]
    return name

In [54]:
def getplayeridfromsn(number,team):
    if team == CUID1:
        name=cuid1_team_meta_df["ObjectId"][cuid1_team_meta_df["ShirtNumber"]==number]
    else:
        name=cuid2_team_meta_df["ObjectId"][cuid2_team_meta_df["ShirtNumber"]==number]
    return name

In [260]:
getplayeridfromsn("11",CUID1)

6    DFL-OBJ-0000O3
Name: ObjectId, dtype: object

In [261]:
getplayername('DFL-OBJ-0000O3')

6    Marco Reus
Name: Name, dtype: object

In [84]:
balldf=positions.loc[:,("BALL","DFL-OBJ-0000XT",slice(None))]

In [251]:
columns = ["X","Y","A"]
Germany = positions.loc[
    # index (rows) - start:end
    :,
    # columns
    (
        CUID1, # team level
        slice(None),   # player level
        columns    # position level
    )
]

In [136]:
playerid="DFL-OBJ-0027G6"
dfa=Germany[CUID1,playerid,'A'][Germany[CUID1,playerid,'A']>5]  ##filter the team df for acceleration higher than 5

In [275]:
##Funktion
def createsprintdf(teamid,playerid,depth):
    dfa=positions[teamid,playerid,'A'][positions[teamid,playerid,'A']>5]
    startframe=[]
    endframe=[]
    for idx, val in enumerate(dfa):
        if idx==0:
             startframe.append(dfa.index.get_level_values('N')[idx])
        if 0<idx and idx<dfa.count()-1:
            if float(dfa.index.get_level_values('N')[idx])-float(dfa.index.get_level_values('N')[idx-1]) > 50:
                endframe.append(dfa.index.get_level_values('N')[idx-1])
                startframe.append(dfa.index.get_level_values('N')[idx])
        if idx==dfa.count()-1:
            endframe.append(dfa.index.get_level_values('N')[idx])
    sprintframes = {'Startframe': startframe,
                 'Endframe': endframe}
    sprintframesdf = pd.DataFrame(sprintframes,columns= ['Startframe', 'Endframe'])
    sprintframesdf["Adj Endframe"]=0
    sprintframesdf["BP"]=0
    sprintframesdf["FinalThird"]=0
    for i in range(len(sprintframesdf.index)):
        for j in range(1000):
            difspeed=cuid1balldf.loc[sprintframesdf.iloc[i,0]+j+1,(teamid,playerid,("S"))].iat[0]-cuid1balldf.loc[sprintframesdf.iloc[i,0]+j,(teamid,playerid,("S"))].iat[0]
            difspeed2=cuid1balldf.loc[sprintframesdf.iloc[i,0]+j+2,(teamid,playerid,("S"))].iat[0]-cuid1balldf.loc[sprintframesdf.iloc[i,0]+j+1,(teamid,playerid,("S"))].iat[0]
            if difspeed<0 or difspeed2<0:
                adjend=sprintframesdf.iloc[i,0]+j+1
                sprintframesdf.iloc[i,2]=adjend
                break   
    if playerid in cuid1_player_IDs:
        for i in range(len(sprintframesdf.index)):
            ballposs=(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),("BALL","DFL-OBJ-0000XT","BallPossession")]==1).sum()
            if ballposs/len(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),("BALL","DFL-OBJ-0000XT","BallPossession")].index)>0.5:
                sprintframesdf["BP"]="Yes"
            else:
                sprintframesdf["BP"]="Yes"
    else:
        for i in range(len(sprintframesdf.index)):
            ballposs=(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),("BALL","DFL-OBJ-0000XT","BallPossession")]==2).sum()
            if ballposs/len(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),("BALL","DFL-OBJ-0000XT","BallPossession")].index)>0.5:
                sprintframesdf["BP"]="Yes"
            else:
                sprintframesdf["BP"]="Yes"
    if playerid in cuid1_player_IDs:
        for i in range(len(sprintframesdf.index)):
            if directionplayframe(sprintframesdf.iloc[i,0],CUID1)=="left to right":
                if (positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),(CUID1,playerid,"X")]>=depth).sum()/len(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),:].index)>0.5:
                    sprintframesdf["FinalThird"]="Yes"
                else:
                    sprintframesdf["FinalThird"]="No"
            if directionplayframe(sprintframesdf.iloc[i,0],CUID1)=="right to left":
                if (positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),(CUID1,playerid,"X")]<=-depth).sum()/len(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),:].index)>0.5:
                    sprintframesdf["FinalThird"]="Yes"
                else:
                    sprintframesdf["FinalThird"]="No"
    return sprintframesdf

In [268]:
createsprintdf(CUID1,'DFL-OBJ-0000O3')

,Startframe,Endframe,Adj Endframe,BP,FinalThird
0,12326,12332,12379,Yes,No
1,12827,12833,12854,Yes,No
2,12918,12921,12945,Yes,No
3,23832,23842,23855,Yes,No
4,25394,25399,25395,Yes,No
5,40613,40620,40627,Yes,No
6,42781,42782,42782,Yes,No
7,44932,44936,44976,Yes,No
8,45115,45117,45127,Yes,No
9,51599,51603,51623,Yes,No


In [137]:
###with A - acceleration
startframe=[]
endframe=[]
for idx, val in enumerate(dfa):
    if idx==0:
         startframe.append(dfa.index.get_level_values('N')[idx])
    if 0<idx and idx<dfa.count()-1:
        if float(dfa.index.get_level_values('N')[idx])-float(dfa.index.get_level_values('N')[idx-1]) > 50:
            endframe.append(dfa.index.get_level_values('N')[idx-1])
            startframe.append(dfa.index.get_level_values('N')[idx])
    if idx==dfa.count()-1:
        endframe.append(dfa.index.get_level_values('N')[idx])

In [196]:
sprintframes = {'Startframe': startframe,
                 'Endframe': endframe}
sprintframesdf = pd.DataFrame(sprintframes,columns= ['Startframe', 'Endframe'])
sprintframesdf["Adj Endframe"]=0
sprintframesdf["BP"]=0
sprintframesdf["FinalThird"]=0
print(sprintframesdf)

    Startframe  Endframe  Adj Endframe  BP  FinalThird
0        11981     11990             0   0           0
1        23291     23294             0   0           0
2        26329     26331             0   0           0
3        34438     34446             0   0           0
4        40531     40539             0   0           0
5        40741     40749             0   0           0
6        41095     41097             0   0           0
7        44928     44935             0   0           0
8        45208     45211             0   0           0
9        51041     51044             0   0           0
10       57844     57852             0   0           0


In [86]:
cuid1balldf=pd.merge(Germany, balldf, left_index=True, right_index=True)

In [112]:
sprintframesdf.iloc[0,0]

10862

In [100]:
(cuid1balldf.loc[(slice(10862,10868),),("BALL","DFL-OBJ-0000XT","BallPossession")]==1).sum()   #anzahl der frames in ballbesitz team 1

7

In [121]:
positions.index[0]

(10000,
 Timestamp('2019-06-11 20:47:01.600000+0200', tz='pytz.FixedOffset(120)'))

In [129]:
def calculateDistance(x1,y1,x2,y2):  
     dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
     return dist  

In [154]:
cuid1balldf.loc[34438,(CUID1,playerid,("S"))].iat[0]

4.61

In [197]:
#test Überprüfung video
##verlängert gegebenenfalls die hohe beschleunigungsphase und erkennt wann der spieler seine höchstgeschwindigkeit erreicht hat
for i in range(len(sprintframesdf.index)):
    for j in range(1000):
        difspeed=cuid1balldf.loc[sprintframesdf.iloc[i,0]+j+1,(CUID1,playerid,("S"))].iat[0]-cuid1balldf.loc[sprintframesdf.iloc[i,0]+j,(CUID1,playerid,("S"))].iat[0]
        difspeed2=cuid1balldf.loc[sprintframesdf.iloc[i,0]+j+2,(CUID1,playerid,("S"))].iat[0]-cuid1balldf.loc[sprintframesdf.iloc[i,0]+j+1,(CUID1,playerid,("S"))].iat[0]
        if difspeed<0 or difspeed2<0:
            adjend=sprintframesdf.iloc[i,0]+j+1
            sprintframesdf.iloc[i,2]=adjend
            break

In [233]:
sprintframesdf

,Startframe,Endframe,Adj Endframe,BP,FinalThird
0,11981,11990,12017,Yes,No
1,23291,23294,23307,Yes,No
2,26329,26331,26330,Yes,No
3,34438,34446,34471,Yes,No
4,40531,40539,40571,Yes,No
5,40741,40749,40742,Yes,No
6,41095,41097,41111,Yes,No
7,44928,44935,44930,Yes,No
8,45208,45211,45219,Yes,No
9,51041,51044,51057,Yes,No


In [195]:
%%capture
i=1
cuid1balldf.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),(CUID1,playerid,("A","S","X","Y"))]

In [194]:
%%capture
for j in range(50):
    print(cuid1balldf.loc[sprintframesdf.iloc[i,0]+j+1,(CUID1,playerid,("S"))].iat[0]-cuid1balldf.loc[sprintframesdf.iloc[i,0]+j,(CUID1,playerid,("S"))].iat[0])

In [204]:
directionplayframe(15215,CUID1)

'left to right'

In [213]:
i=1
positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),("BALL","DFL-OBJ-0000XT","BallPossession")]

N      T                               
23291  2019-06-11 20:55:53.240000+02:00    1
23292  2019-06-11 20:55:53.280000+02:00    1
23293  2019-06-11 20:55:53.320000+02:00    1
23294  2019-06-11 20:55:53.360000+02:00    1
23295  2019-06-11 20:55:53.400000+02:00    1
23296  2019-06-11 20:55:53.440000+02:00    1
23297  2019-06-11 20:55:53.480000+02:00    1
23298  2019-06-11 20:55:53.520000+02:00    1
23299  2019-06-11 20:55:53.560000+02:00    1
23300  2019-06-11 20:55:53.600000+02:00    1
23301  2019-06-11 20:55:53.640000+02:00    1
23302  2019-06-11 20:55:53.680000+02:00    1
23303  2019-06-11 20:55:53.720000+02:00    1
23304  2019-06-11 20:55:53.760000+02:00    1
23305  2019-06-11 20:55:53.800000+02:00    1
23306  2019-06-11 20:55:53.840000+02:00    1
23307  2019-06-11 20:55:53.880000+02:00    2
Name: (BALL, DFL-OBJ-0000XT, BallPossession), dtype: int32

In [216]:
##Bestimmt ob in eigenem Ballbesitz während Angriff
if playerid in cuid1_player_IDs:
    for i in range(len(sprintframesdf.index)):
        ballposs=(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),("BALL","DFL-OBJ-0000XT","BallPossession")]==1).sum()
        if ballposs/len(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),("BALL","DFL-OBJ-0000XT","BallPossession")].index)>0.5:
            sprintframesdf["BP"]="Yes"
        else:
            sprintframesdf["BP"]="Yes"
else:
    for i in range(len(sprintframesdf.index)):
        ballposs=(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),("BALL","DFL-OBJ-0000XT","BallPossession")]==2).sum()
        if ballposs/len(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),("BALL","DFL-OBJ-0000XT","BallPossession")].index)>0.5:
            sprintframesdf["BP"]="Yes"
        else:
            sprintframesdf["BP"]="Yes"

In [232]:
##Finalthird Überprüfung 
if playerid in cuid1_player_IDs:
    for i in range(len(sprintframesdf.index)):
        if directionplayframe(sprintframesdf.iloc[i,0],CUID1)=="left to right":
            if (positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),(CUID1,playerid,"X")]>=35).sum()/len(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),:].index)>0.5:
                sprintframesdf["FinalThird"]="Yes"
            else:
                sprintframesdf["FinalThird"]="No"
        if directionplayframe(sprintframesdf.iloc[i,0],CUID1)=="right to left":
            if (positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),(CUID1,playerid,"X")]<=-35).sum()/len(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),:].index)>0.5:
                sprintframesdf["FinalThird"]="Yes"
            else:
                sprintframesdf["FinalThird"]="No"

In [242]:
i=7
(positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),(CUID1,playerid,"X")]>=35).sum()

0

In [243]:
positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2]),),(CUID1,playerid,"X")]

N      T                               
44928  2019-06-11 21:10:18.720000+02:00    22.59
44929  2019-06-11 21:10:18.760000+02:00    22.58
44930  2019-06-11 21:10:18.800000+02:00    22.60
Name: (DFL-CLU-000N8Y, DFL-OBJ-0027G6, X), dtype: float64

In [231]:
positions.loc[(slice(sprintframesdf.iloc[i,0],sprintframesdf.iloc[i,2])),:]

TeamId                                           BALL                 \
PersonId                               DFL-OBJ-0000XT                  
Position                                            A BallPossession   
N     T                                                                
23291 2019-06-11 20:55:53.240000+02:00            0.0              1   
23292 2019-06-11 20:55:53.280000+02:00            0.0              1   
23293 2019-06-11 20:55:53.320000+02:00            0.0              1   
23294 2019-06-11 20:55:53.360000+02:00            0.0              1   
23295 2019-06-11 20:55:53.400000+02:00            0.0              1   
23296 2019-06-11 20:55:53.440000+02:00            0.0              1   
23297 2019-06-11 20:55:53.480000+02:00            0.0              1   
23298 2019-06-11 20:55:53.520000+02:00            0.0              1   
23299 2019-06-11 20:55:53.560000+02:00            0.0              1   
23300 2019-06-11 20:55:53.600000+02:00            0.0              1   
23301 2019-06-11 20:55:53.640000+02:00            0.0              1   
23302 2019-06-11 20:55:53.680000+02:00            0.0              1   
23303 2019-06-11 20:55:53.720000+02:00            0.0              1   
23304 2019-06-11 20:55:53.760000+02:00            0.0              1   
23305 2019-06-11 20:55:53.800000+02:00            0.0              1   
23306 2019-06-11 20:55:53.840000+02:00            0.0              1   
23307 2019-06-11 20:55:53.880000+02:00            0.0              2   

TeamId                                                                     \
PersonId                                                                    
Position                               BallStatus      D  M      S      X   
N     T                                                                     
23291 2019-06-11 20:55:53.240000+02:00          1  41.50  9  39.70  25.76   
23292 2019-06-11 20:55:53.280000+02:00          1  42.60  9  38.35  25.94   
23293 2019-06-11 20:55:53.320000+02:00          1  28.64  9  38.33  26.05   
23294 2019-06-11 20:55:53.360000+02:00          1  34.37  9  32.17  26.18   
23295 2019-06-11 20:55:53.400000+02:00          1  36.43  9  31.23  26.31   
23296 2019-06-11 20:55:53.440000+02:00          1  35.60  9  29.97  26.43   
23297 2019-06-11 20:55:53.480000+02:00          1  36.00  9  32.08  26.56   
23298 2019-06-11 20:55:53.520000+02:00          1  34.77  9  32.39  26.68   
23299 2019-06-11 20:55:53.560000+02:00          1  32.32  9  31.97  26.81   
23300 2019-06-11 20:55:53.600000+02:00          1  33.95  9  31.04  26.93   
23301 2019-06-11 20:55:53.640000+02:00          1  31.50  9  29.74  27.05   
23302 2019-06-11 20:55:53.680000+02:00          1  34.90  9  29.30  27.15   
23303 2019-06-11 20:55:53.720000+02:00          1  29.45  9  31.01  27.26   
23304 2019-06-11 20:55:53.760000+02:00          1  28.23  9  28.71  27.36   
23305 2019-06-11 20:55:53.800000+02:00          1  29.53  9  27.72  27.45   
23306 2019-06-11 20:55:53.840000+02:00          1  36.08  9  26.39  27.45   
23307 2019-06-11 20:55:53.880000+02:00          1  74.32  9  29.30  27.84   

TeamId                                             DFL-CLU-000N8Y  ...  \
PersonId                                           DFL-OBJ-000191  ...   
Position                                   Y     Z              A  ...   
N     T                                                            ...   
23291 2019-06-11 20:55:53.240000+02:00  3.99  0.00          -1.18  ...   
23292 2019-06-11 20:55:53.280000+02:00  3.60  0.11          -0.85  ...   
23293 2019-06-11 20:55:53.320000+02:00  3.34  0.10          -0.48  ...   
23294 2019-06-11 20:55:53.360000+02:00  3.02  0.16          -0.09  ...   
23295 2019-06-11 20:55:53.400000+02:00  2.68  0.21           0.26  ...   
23296 2019-06-11 20:55:53.440000+02:00  2.34  0.26           0.55  ...   
23297 2019-06-11 20:55:53.480000+02:00  2.01  0.31           0.74  ...   
23298 2019-06-11 20:55:53.520000+02:00  1.68  0.36     

In [248]:
len(sprintframesdf[(sprintframesdf.BP=="Yes") & (sprintframesdf.FinalThird=="Yes")].index)

0

In [258]:
cuid1players=CUID1df.columns.get_level_values(level=1).unique()

In [279]:
###Zusammenspiel aller Codeschnipsel für alle Spieler
numofsprints=0
for player in cuid1players:
    currentdf=createsprintdf(CUID1,player,17.5)
    numofsprints=numofsprints+len(currentdf[(currentdf.BP=="Yes") & (currentdf.FinalThird=="Yes")].index)
print(numofsprints)

13
24
24
36
36
36
49
57
74
88
88


In [273]:
numofsprints

0